# TinyAleph: Quaternions and 3D Rotations

This notebook explores quaternions and hypercomplex algebras in TinyAleph:

- **Quaternions (ℍ)**: 4D numbers for 3D rotations
- **Quaternion arithmetic**: Non-commutative multiplication
- **3D Rotations**: Representing and composing rotations
- **Octonions (𝕆)**: 8D extension (non-associative)
- **Sedenions (𝕊)**: 16D extension (contains zero divisors)

In [ ]:
import sys
sys.path.insert(0, '..')

from tinyaleph.core.quaternion import Quaternion
from tinyaleph.core.hypercomplex import Octonion, Sedenion
import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

print("TinyAleph Quaternion Module")

## 1. Quaternion Basics

A quaternion is: $q = w + xi + yj + zk$ where $i^2 = j^2 = k^2 = ijk = -1$

In [ ]:
# Create quaternions
q1 = Quaternion(1, 2, 3, 4)
q2 = Quaternion(0.5, -1, 0.5, 2)

print(f"q1 = {q1}")
print(f"q2 = {q2}")
print(f"\nComponents: w={q1.w}, i={q1.i}, j={q1.j}, k={q1.k}")

In [ ]:
# Special quaternions
print(f"Zero: {Quaternion.zero()}")
print(f"One:  {Quaternion.one()}")
print(f"i:    {Quaternion.i()}")
print(f"j:    {Quaternion.j()}")
print(f"k:    {Quaternion.k()}")

In [ ]:
# Quaternion arithmetic
print(f"q1 + q2 = {q1 + q2}")
print(f"q1 - q2 = {q1 - q2}")
print(f"q1 × q2 = {q1 * q2}")
print(f"q1 ÷ q2 = {q1 / q2}")

## 2. Non-Commutativity

Quaternion multiplication is NOT commutative: $ij = k$ but $ji = -k$

In [ ]:
i = Quaternion.i()
j = Quaternion.j()
k = Quaternion.k()

print(f"i² = {i * i}  (should be -1)")
print(f"j² = {j * j}  (should be -1)")
print(f"k² = {k * k}  (should be -1)")
print(f"\nij = {i * j}  (should be k)")
print(f"ji = {j * i}  (should be -k)")

## 3. Properties

In [ ]:
q = Quaternion(1, 2, 3, 4)

print(f"q = {q}")
print(f"|q| (norm) = {q.norm():.6f}")
print(f"q̄ (conjugate) = {q.conjugate()}")
print(f"q⁻¹ (inverse) = {q.inverse()}")
print(f"\nq × q⁻¹ = {q * q.inverse()}  (should be ~1)")

In [ ]:
# Normalization
q_unit = q.normalize()
print(f"Original norm: {q.norm():.6f}")
print(f"Normalized: {q_unit}")
print(f"Unit norm: {q_unit.norm():.6f}")

## 4. 3D Rotations

A rotation of angle θ around axis (x, y, z):
$q = \cos(\theta/2) + \sin(\theta/2)(xi + yj + zk)$

In [ ]:
# 90° rotation around z-axis
rotation_z = Quaternion.from_axis_angle(0, 0, 1, math.pi/2)
point = (1, 0, 0)
rotated = rotation_z.rotate_point(point)

print(f"Point: {point}")
print(f"After 90° z-rotation: ({rotated[0]:.4f}, {rotated[1]:.4f}, {rotated[2]:.4f})")
print(f"Expected: (0, 1, 0)")

In [ ]:
# Visualize rotation
fig = plt.figure(figsize=(10, 5))

points = [(1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)]
rotated_points = [rotation_z.rotate_point(p) for p in points]

ax1 = fig.add_subplot(121, projection='3d')
for p in points:
    ax1.scatter(*p, c='blue', s=100)
ax1.set_title('Original')
ax1.set_xlim([-1.5, 1.5]); ax1.set_ylim([-1.5, 1.5]); ax1.set_zlim([-1.5, 1.5])

ax2 = fig.add_subplot(122, projection='3d')
for p in rotated_points:
    ax2.scatter(*p, c='red', s=100)
ax2.set_title('After 90° Z-rotation')
ax2.set_xlim([-1.5, 1.5]); ax2.set_ylim([-1.5, 1.5]); ax2.set_zlim([-1.5, 1.5])

plt.tight_layout()
plt.show()

## 5. Composing Rotations

Composing rotations is just multiplication: $R_2 \circ R_1 = q_2 \cdot q_1$

In [ ]:
rot_x_90 = Quaternion.from_axis_angle(1, 0, 0, math.pi/2)
rot_y_90 = Quaternion.from_axis_angle(0, 1, 0, math.pi/2)

combined = rot_y_90 * rot_x_90  # x first, then y

point = (1, 0, 0)
step1 = rot_x_90.rotate_point(point)
step2 = rot_y_90.rotate_point(step1)
combined_result = combined.rotate_point(point)

print(f"Step-by-step: ({step2[0]:.4f}, {step2[1]:.4f}, {step2[2]:.4f})")
print(f"Combined:     ({combined_result[0]:.4f}, {combined_result[1]:.4f}, {combined_result[2]:.4f})")

## 6. SLERP (Spherical Linear Interpolation)

In [ ]:
rot_start = Quaternion.from_axis_angle(0, 0, 1, 0)
rot_end = Quaternion.from_axis_angle(0, 0, 1, math.pi)

point = (1, 0, 0)
print("Interpolating from 0° to 180°:")
for t in [0.0, 0.25, 0.5, 0.75, 1.0]:
    q = rot_start.slerp(rot_end, t)
    rotated = q.rotate_point(point)
    print(f"  t={t:.2f}: ({rotated[0]:.3f}, {rotated[1]:.3f}, {rotated[2]:.3f})")

## 7. Matrix and Euler Conversion

In [ ]:
q = Quaternion.from_axis_angle(0, 0, 1, math.pi/4)
matrix = q.to_matrix()

print("Rotation matrix:")
for row in matrix:
    print(f"  [{row[0]:8.4f}  {row[1]:8.4f}  {row[2]:8.4f}]")

In [ ]:
euler = q.to_euler()
print(f"Euler angles (degrees):")
print(f"  Roll:  {math.degrees(euler[0]):.2f}°")
print(f"  Pitch: {math.degrees(euler[1]):.2f}°")
print(f"  Yaw:   {math.degrees(euler[2]):.2f}°")

## 8. Octonions (8D) and Sedenions (16D)

In [ ]:
# Octonions - 8 dimensional, non-associative
o1 = Octonion([1, 0, 0, 0, 0, 0, 0, 0])
o2 = Octonion([0, 1, 0, 0, 0, 0, 0, 0])

print(f"o1 = {o1}")
print(f"o2 = {o2}")
print(f"o1 × o2 = {o1 * o2}")

In [ ]:
# Sedenions - 16 dimensional, has zero divisors
s = Sedenion([1] + [0]*15)
print(f"Sedenion dimension: 16")
print(f"Identity: {s}")
print(f"Norm: {s.norm():.4f}")

## Summary

- **Quaternions** represent 3D rotations without gimbal lock
- **Non-commutative**: Order of multiplication matters
- **Unit quaternions** form the 3-sphere S³
- **SLERP** gives smooth interpolation
- **Cayley-Dickson** construction: ℝ → ℂ → ℍ → 𝕆 → 𝕊